In [1]:
import networkx as nx
import matplotlib.pyplot as plt

from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister
from qiskit_aer import Aer
from qiskit.circuit import Parameter
from qiskit.visualization import plot_histogram

import numpy as np
from scipy.optimize import minimize

In [2]:
# dij需求i到设施j的成本
d = [[1, 2], [1, 2]]
n = 2   # 两个设施点
m = 2   # 两个需求点
# d = [[1, 2], [3, 4], [5, 6]]
# n = 2   # 两个设施点
# m = 3   # 三个需求点
num_qubits = n + 2 * n * m

# gi设施i的建设成本
g = [2, 1]


In [3]:
penalty = 25
depth = 6
params = np.ones(depth * 2)

In [4]:
# dtype=np.complex128
def add_in_target(num_qubits, target_qubit, gate=np.array([[1, 0],[0, -1]])):
    H = np.eye(2 ** (target_qubit))
    H = np.kron(H, gate)
    H = np.kron(H, np.eye(2 ** (num_qubits - 1 - target_qubit)))
    return H
def add_in_target2(num_qubits, target_qubit, gate=(np.array([[1, 0],[0, -1]])-np.eye(2))/2):
    H = np.eye(2 ** (target_qubit))
    H = np.kron(H, gate)
    H = np.kron(H, np.eye(2 ** (num_qubits - 1 - target_qubit)))
    return H

def generate_Hp(n, m, d, g):
    # 初始化 Hp 矩阵为零矩阵
    # print(num_qubits)
    Hp = np.zeros((2**num_qubits, 2**num_qubits))
    for i in range(m):
        for j in range(n):
            Hp += d[i][j] * add_in_target2(num_qubits, n * (1 + i) + j)
    
    for j in range(n):
        Hp +=  g[j] * add_in_target2(num_qubits, j)

    for i in range(m):
        Ht = np.zeros((2**num_qubits, 2**num_qubits))
        for j in range(n):
            Ht += (np.eye(2**num_qubits) - add_in_target(num_qubits, n * (1 + i) + j)) / 2
        Ht -= np.eye(2**num_qubits)
        Hp += -penalty * Ht @ Ht
    
    for i in range(m):
        for j in range(n):
            Ht = (np.eye(2**num_qubits) - add_in_target(num_qubits, n * (1 + i) + j)) / 2 + (np.eye(2**num_qubits) - add_in_target(num_qubits, n * (1 + m + i) + j)) / 2 + (np.eye(2**num_qubits) - add_in_target(num_qubits, j)) / 2
            Hp += -penalty * Ht @ Ht

    return Hp

In [5]:
from scipy.linalg import expm
def build_circ(n, m, d, g, params):
  qc = QuantumCircuit(num_qubits)
  beta = params[:depth + 1]
  gamma = params[depth:]
  for i in range(num_qubits):
    qc.h(i)
  for dp in range(depth):
    qc.unitary(expm(-1j * gamma[dp] * generate_Hp(n, m, d, g)), range(num_qubits))
    for i in range(num_qubits):
      qc.rx(beta[dp], i)
  qc.measure_all()
  return qc

In [6]:
def cost_function(x):
  num = [int(char) for char in x]
  num = num[::-1]
  C = 0
  for i in range(m):
    for j in range(n):
      C += d[i][j] * num[n * (1 + i) + j]
      
  for j in range(n):
    C += g[j] * num[j]

  for i in range(m):
    t = 0
    for j in range(n):
      t += num[n * (1 + i) + j]
    C += penalty * (t - 1)**2

  for i in range(m):
    for j in range(n):
      C += penalty * (num[n * (1 + i) + j] + num[n * (1 + m + i) + j] - num[j]) ** 2
  return C

def compute_expectation(counts):
  EV = 0
  total_count = 0
  for x, count in counts.items():
    C = cost_function(x)
    EV += C*count
    total_count += count

  return EV/total_count


def expectation_from_sample(shots = 2000):
  backend = Aer.get_backend('qasm_simulator')
  backend.shots = shots

  def execute_circ(theta):
    qc = build_circ(n, m, d, g, theta)
    counts = backend.run(qc, seed_simulator=10, shots=shots).result().get_counts()
    return compute_expectation(counts)
  
  return execute_circ

In [7]:
from scipy.optimize import minimize
import numpy as np

expectation = expectation_from_sample()

def callback(x):
    global iteration_count
    iteration_count += 1
    if iteration_count % 10 == 0:
        print(f"Iteration {iteration_count}, Result: {expectation(x)}")

# 设定最大迭代次数
max_iterations = 1000

# 初始化迭代计数器
iteration_count = 0

# 使用 COBYLA 方法进行最小化，并设置 callback 函数
res = minimize(expectation, params, method='COBYLA', options={'maxiter': max_iterations}, callback=callback)

# 输出最终结果
print("Final Result:", res)

Iteration 10, Result: 123.0715
Iteration 20, Result: 81.5905
Iteration 30, Result: 93.4675
Iteration 40, Result: 132.046
Iteration 50, Result: 83.6695
Iteration 60, Result: 85.723
Iteration 70, Result: 90.7285
Iteration 80, Result: 77.9605
Iteration 90, Result: 78.4755
Iteration 100, Result: 78.9495
Iteration 110, Result: 77.4135
Iteration 120, Result: 76.699
Final Result:  message: Optimization terminated successfully.
 success: True
  status: 1
     fun: 78.8995
       x: [ 2.035e+00  9.815e-01  9.927e-01  1.684e+00  9.923e-01
            9.942e-01  8.730e-01  9.974e-01  9.984e-01  8.658e-01
            9.981e-01  9.985e-01]
    nfev: 124
   maxcv: 0.0


In [8]:
from numpy.lib.utils import source
backend = Aer.get_backend('aer_simulator')
backend.shots = 100000

shots=100000
qc_res = build_circ(n, m, d, g, params=res.x)

counts = backend.run(qc_res, seed_simulator=10, shots = shots).result().get_counts()
# plot_histogram(counts)

In [9]:
sorted_counts = sorted(counts, key=counts.get, reverse=True)
print("\n----------------- Full result ---------------------")
print("selection\t\tprobability\tvalue")
print("---------------------------------------------------")
for x in sorted_counts[:20]:
  print(x, "{:.1f}%".format(counts[x] / shots * 100), cost_function(x))


----------------- Full result ---------------------
selection		probability	value
---------------------------------------------------
0110101111 18.8% 58
0100101111 6.5% 33
0110001111 6.5% 106
1100101111 5.7% 58
1110001111 5.6% 81
1110101111 5.5% 83
1100001111 5.0% 56
0110001101 2.2% 155
0100100111 2.2% 31
0010101010 1.1% 30
0010101111 1.0% 83
0100001101 0.9% 80
0100000111 0.9% 79
0010101011 0.9% 82
0100001111 0.9% 81
0010101110 0.8% 81
0111101011 0.8% 32
0110111110 0.7% 207
0101101111 0.6% 58
0110011111 0.6% 107
